In this task, you will train a state-of-the-art semantic segmentation model known as **[HRNet](https://arxiv.org/pdf/1908.07919) (*High Resolution Net*)**. They have demonstrated superior performance compared to the widely recognized and commonly utilized models discussed in class, including:
* **FCN**
* **SegNet**
* **PSPNet**
* **U-Net**

##**Install the necessary libraries**

In [1]:
import torch
print(torch.__version__)

2.4.0+cpu


In [1]:
import torchvision
print(torchvision.__version__)

0.19.0


In [2]:
!pip install gdown

In [3]:
!pip install imagecodecs
!pip install pickle5
!pip install timm
!pip install tensorboardX
!pip install yacs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 41.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=125856 sha256=5bfaaf3cf5bdc3a5d5969c7e92326ad6596aced03217abee7811bbc92c10e0c8
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [5]:
![[ -d ./hrnetv2_sem_seg ]]
!rm -rf ./hrnetv2_sem_seg
!git clone https://github.com/alimoorreza/hrnetv2_sem_seg

Cloning into 'hrnetv2_sem_seg'...
remote: Enumerating objects: 2325, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 2325 (delta 193), reused 432 (delta 175), pack-reused 1866 (from 1)
Receiving objects: 100% (2325/2325), 181.27 MiB | 52.44 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [6]:
!mkdir -p hrnetv2_sem_seg/log
!mkdir -p hrnetv2_sem_seg/output
!mkdir -p hrnetv2_sem_seg/init_model
!mkdir -p hrnetv2_sem_seg/dataset

##**Download the [UWSv1 dataset](https://drive.google.com/uc?id=155Vg3iOTK3aed11LXC2lFQBxhdimfuZB)**
* This dataset comprises 574 images featuring 21 underwater animal categories and background objects such as *water and rock*. Pixel-wise annotations were provided by in-house annotators from Indiana University and Drake University, encompassing 29 total categories. The dataset is divided into training (461 images) and testing (113 images) sets using an 80:20 random split.

In [7]:
!gdown https://drive.google.com/uc?id=155Vg3iOTK3aed11LXC2lFQBxhdimfuZB

Downloading...
From (original): https://drive.google.com/uc?id=155Vg3iOTK3aed11LXC2lFQBxhdimfuZB
From (redirected): https://drive.google.com/uc?id=155Vg3iOTK3aed11LXC2lFQBxhdimfuZB&confirm=t&uuid=9cf13dfd-38d5-470c-bbd2-1242f98ba8c1
To: /kaggle/working/uw_sem_seg_dataset.zip
100%|█████████████████████████████████████████| 571M/571M [00:03<00:00, 173MB/s]


In [8]:
!unzip /kaggle/working/uw_sem_seg_dataset.zip  -d hrnetv2_sem_seg/dataset

Archive:  /kaggle/working/uw_sem_seg_dataset.zip
   creating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/
  inflating: hrnetv2_sem_seg/dataset/__MACOSX/._uw_sem_seg_dataset  
   creating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/train/
  inflating: hrnetv2_sem_seg/dataset/__MACOSX/uw_sem_seg_dataset/._train  
  inflating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/._train  
   creating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/validation/
  inflating: hrnetv2_sem_seg/dataset/__MACOSX/uw_sem_seg_dataset/._validation  
  inflating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/._validation  
   creating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/train/images/
  inflating: hrnetv2_sem_seg/dataset/__MACOSX/uw_sem_seg_dataset/train/._images  
  inflating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/train/._images  
   creating: hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/train/labels/
  inflating: hrnetv2_sem_seg/dataset/__MACOSX/uw_sem_seg_dataset/train/._labels  
  inflating: hrnetv2_sem_

##**Download the pretrained weights for hrnet-v2**

In [9]:
!gdown https://drive.google.com/uc?id=1CHOKZcn6TnThoajYmfwjJ9g7Q4SRW4k4
!mv hrnetv2_w48_imagenet_pretrained.pth hrnetv2_sem_seg/init_model/


Downloading...
From (original): https://drive.google.com/uc?id=1CHOKZcn6TnThoajYmfwjJ9g7Q4SRW4k4
From (redirected): https://drive.google.com/uc?id=1CHOKZcn6TnThoajYmfwjJ9g7Q4SRW4k4&confirm=t&uuid=5e5d5ebc-8e23-4e20-92b9-86289b9bd7f0
To: /kaggle/working/hrnetv2_w48_imagenet_pretrained.pth
100%|█████████████████████████████████████████| 311M/311M [00:01<00:00, 201MB/s]


##**Cleanup folder names such as .DS_store**

In [10]:
!find ./hrnetv2_sem_seg -name "._*" -delete
!find ./hrnetv2_sem_seg/dataset/ -name "__MACOSX" -exec rm -rf {} +

##**Run the training script**

In [ ]:
!python3 ./hrnetv2_sem_seg/tools/train_hrnet_v2_uws.py \
    --cfg /kaggle/working/hrnetv2_sem_seg/experiments/hrnet/uws_training_hrnet_v2_train_KAGGLE.yaml


Seeding with 304
=> creating /kaggle/working/hrnetv2_sem_seg/output/UWS3/uws_training_hrnet_v2_train_KAGGLE
=> creating /kaggle/working/hrnetv2_sem_seg/log/UWS3/seg_hrnet/uws_training_hrnet_v2_train_KAGGLE_2024-12-04-23-27
Namespace(cfg='/kaggle/working/hrnetv2_sem_seg/experiments/hrnet/uws_training_hrnet_v2_train_KAGGLE.yaml', seed=304, opts=[])
AUTO_RESUME: False
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  DATASET: UWS3
  EXTRA_TRAIN_SET: 
  NUM_CLASSES: 29
  ROOT: /kaggle/working/hrnetv2_sem_seg/dataset/uw_sem_seg_dataset/
  TEST_SET: validation
  TRAIN_SET: train
DEBUG:
  DEBUG: False
  SAVE_BATCH_IMAGES_GT: False
  SAVE_BATCH_IMAGES_PRED: False
  SAVE_HEATMAPS_GT: False
  SAVE_HEATMAPS_PRED: False
GPUS: (0,)
LOG_DIR: /kaggle/working/hrnetv2_sem_seg//log
LOSS:
  BALANCE_WEIGHTS: [1]
  CLASS_BALANCE: False
  OHEMKEEP: 131072
  OHEMTHRES: 0.9
  USE_OHEM: True
MODEL:
  ALIGN_CORNERS: True
  EXTRA:
    FINAL_CONV_KERNEL: 1
    STAGE1:
      BLOCK: BOTTLEN

# __Task 2.1:__ (3 points)
### **Training HRNet semantic segmentation model using UWSv1 dataset.**
- **Step 1:** Finish training [HRNet v2 model](https://arxiv.org/pdf/1908.07919)
- **Step 2:** The performance of the semantic segmentation task is evaluated using the **mean Intersection over Union (mIoU)** metric. Please examine the outputs and identify the highest **mIoU** achieved during training. The mIoU may range between 40.45% and 60%. Kindly report the best **mIoU** obtained.
- **Step 3:**  Please identify the epoch number corresponding to the best **mIoU**. You will need to review the log to determine the epoch that yields the highest value.

You may proceed to **Part 3** of this assignment upon completion of the current task.